In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from utils import *
from plot_functions import *

# Set Experiment SetUp

In [ ]:
# Set total arterial compliance (C_PPM) as the prediction variable
prediction_variable = 'C_PPM'  
variable_unit = 'mL/mmHg'

# Select regression model, e.g., 'LR1','LR2','ANN1','ANN2','ANN3','ANN4'
model_selection = 'ANN1' 

# Enable addition of noise and select noise level by setting an snr_dB value
noise_mode = False
snr_dB = 30

# Enable figure saving
save_figure_boolean = False

# Enable printing results during training
verbose = False

# Set the experiment type
experiment_type = 'invivo'

# Load & Prepare Data

In [ ]:
dataset,regressor = prepare_dataset(model_selection,prediction_variable,verbose,noise_mode,snr_dB)
dataset.head()

In [ ]:
if 'ANN' in model_selection: 
    scaled_dataset = scale_data(dataset)

# Regression Analysis

### Train/validation/test split

In [ ]:
X,y = split_features_target(scaled_dataset)

indices = np.arange(len(X))
X_train_val, X_test, y_train_val, y_test, train_val_indices, test_indices = train_test_split(X, y, indices, test_size=0.105, random_state=42)

indices2 = np.arange(len(X_train_val))
X_train, X_val, y_train, y_val, train_indices, val_indices = train_test_split(X_train_val, y_train_val,indices2,test_size=0.11, random_state=42) 

print('The train set size is:', X_train.shape)
print('The validation set size is:', X_val.shape)
print('The test set size is:', X_test.shape)

### Epoch Selection with Early Stopping

In [ ]:
selected_batch_size = 200
 
# optimal_no_epochs = find_optimal_no_epochs(selected_batch_size, scaled_dataset, X_train, X_val, y_train, y_val, prediction_variable, verbose)

### Model training and testing

In [ ]:
model, y_pred = select_regression_model(X_train, X_test, y_train, model_selection, selected_batch_size, verbose)

In [ ]:
if 'ANN' in model_selection: 
        y_pred = rescale_values(y_pred, prediction_variable, dataset)
        y_test = rescale_values(y_test, prediction_variable, dataset)

### Print results

In [ ]:
print_results(y_test, y_pred, variable_unit)

### Plot results

In [ ]:
plot_data(y_test,y_pred,experiment_type,prediction_variable,model_selection,save_figure_boolean)

# Permutation feature importances

In [ ]:
# Define the model
model, y_pred = artificial_neural_network(selected_batch_size, 118, X_train, X_test, y_train, verbose)

# Define parameters
num_iterations = 20
rmse_differences = []
importance_sum = np.zeros(X_train.shape[1])

# Run the permutation feature importance calculation multiple times
for _ in range(num_iterations):
    importances = permutation_importances(model, X_train, y_train, mean_squared_error)
    baseline_rmse = mean_squared_error(y_train, model.predict(X_train))
    new_rmse = baseline_rmse + importances
    rmse_difference = new_rmse - baseline_rmse
    rmse_differences.append(rmse_difference)
    importance_sum += importances

# Calculate mean and standard deviation of RMSE differences
mean_rmse_difference = np.mean(rmse_differences)
std_dev_rmse_difference = np.std(rmse_differences)

# Calculate average importance
average_importance = importance_sum / num_iterations

# Create a list of tuples with feature index and average importance
feature_importance_tuples = list(enumerate(average_importance))

# Sort the list by importance in ascending order
feature_importance_tuples.sort(key=lambda x: x[1])

# Print the importances in ascending order
for i, importance in feature_importance_tuples:
    print(f'Feature {i+1}: {importance}')

# Print the results
print(f"\nMean Increase in RMSE: {mean_rmse_difference}")
print(f"Standard Deviation of Increase in RMSE: {std_dev_rmse_difference}")